# Data Preprocessing


##### Downloading dataset


In [ ]:
import opendatasets as od
import pandas as pd

od.download(
    'https://www.kaggle.com/datasets/shariful07/student-mental-health/data')

dataset = pd.read_csv("student-mental-health\\Student Mental health.csv")
dataset.head()

In [ ]:
# Dropping timestamp column
dataset = dataset.drop("Timestamp", axis=1)

In [ ]:
# Renaming columns
dataset.rename(columns={'OldColumnName1': 'NewColumnName1', 'OldColumnName2': 'NewColumnName2'}, inplace=True)

In [ ]:
dataset.head()

In [ ]:
# Processing gender column
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
# Filling empty cells with value "Female"
dataset['Choose your gender'].fillna('Female', inplace=True)
dataset['Choose your gender'] = label_encoder.fit_transform(
    dataset['Choose your gender'])

In [ ]:
# Processing age column
mean_age = dataset['Age'].mean()
dataset['Age'].fillna(mean_age, inplace=True)
# Round the mean value to the nearest integer (optional)
mean_age = round(mean_age)
# Convert the 'Age' column to integers
dataset['Age'] = dataset['Age'].astype(int)

In [ ]:
# Processing course column

import openai
import time

# Set your OpenAI GPT-3 API key
openai.api_key = ''


def categorize_course_with_gpt3(course):
    # Construct a prompt for GPT-3 to perform a web search
    prompt = f"Select the field of this course {course} from these given fields. Do not reply anything else than the given feilds. Reply only the strings of field:\n" \
             "Arts and Humanities\n" \
             "Social Sciences\n" \
             "Natural Sciences\n" \
             "Engineering and Technology\n" \
             "Health Sciences\n" \
             "Business and Economics\n" \
             "Law\n" \
             "Religious Studies\n" \
             "Agriculture and Environmental Studies\n" \
             "Interdisciplinary Studies\n" \
             "Communication and Media\n" \
             "Computer and Information Sciences\n" \
             "Physical Education and Sports Sciences\n" \
             "Mathematics and Statistics\n" \
             "Public Policy and Administration\n" \
             "Interdisciplinary and General Education\n"

    # Make an API call to GPT-3 using the chat completion endpoint
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150  # Adjust as needed
    )
    return response["choices"][0]["message"]["content"]


# Set the rate limit variables
requests_limit = 3
wait_time = 60

# Iterate over the DataFrame and update values
for index, row in dataset.iterrows():
    current_value = row['What is your course?']
    if pd.notna(current_value):
        updated_value = categorize_course_with_gpt3(current_value)
        dataset.at[index, 'What is your course?'] = updated_value
        print(updated_value)
    else:
        dataset.at[index, 'What is your course?'] = "General Studies"
        print("General Studies")

    # Check if 3 requests have been made and introduce a wait
    if (index + 1) % requests_limit == 0 and index != 0:
        print(f"Waiting for {wait_time} seconds...")
        time.sleep(wait_time)

In [ ]:
# Processing course column -- continue

dataset['What is your course?'] = label_encoder.fit_transform(
    dataset['What is your course?'])

In [ ]:
# Processing Your current year of Study column

label_mapping_year = {
    "year 1": 0,
    "year 2": 1,
    "year 3": 2,
    "year 4": 3
}

# Clean up column values by converting to lowercase and removing leading and trailing spaces
dataset['Your current year of Study'] = dataset['Your current year of Study'].str.lower().str.strip()

# Apply label encoding using a for loop with a check for matching characters
dataset['Your current year of Study'] = [next((label_mapping_year[label] for label in label_mapping_year if all(char in year for char in label)), None) for year in dataset['Your current year of Study']]


In [ ]:
# Processing course CGPA column

label_mapping = {
    "0-1.99": 0,
    "2.00-2.49": 1,
    "2.50-2.99": 2,
    "3.00-3.49": 3,
    "3.5-4.00": 4
}

# Clean up column values by removing leading and trailing spaces
dataset['What is your CGPA?'] = dataset['What is your CGPA?'].str.strip()

# Apply label encoding using a for loop with a check for matching characters
dataset['What is your CGPA?'] = [next((label_mapping[label] for label in label_mapping if all(
    char in cgpa for char in label)), None) for cgpa in dataset['What is your CGPA?']]

In [ ]:
# Processing martial status column

dataset['Marital status'] = label_encoder.fit_transform(
    dataset['Marital status'])

In [ ]:
# Processing Do you have Depression? column

dataset['Do you have Depression?'] = label_encoder.fit_transform(
    dataset['Do you have Depression?'])

In [ ]:
# Processing Do you have Anxiety? column

dataset['Do you have Anxiety?'] = label_encoder.fit_transform(
    dataset['Do you have Anxiety?'])

In [ ]:
# Processing Do you have Panic attack? column

dataset['Do you have Panic attack?'] = label_encoder.fit_transform(
    dataset['Do you have Panic attack?'])

In [ ]:
# Processing Did you seek any specialist for a treatment? column

dataset['Did you seek any specialist for a treatment?'] = label_encoder.fit_transform(
    dataset['Did you seek any specialist for a treatment?'])

In [ ]:
import seaborn as sns
import pandas as pd

dataset = pd.read_csv("new_file.csv")
sns.pairplot(dataset)

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode

dataset.sort_values(by= "Age" ,inplace=True)
plt.figure(dpi=200)
fig = px.bar(dataset, x="Age", y="What is your course?", orientation='h',color='Do you have Depression?')
fig.show()

In [ ]:
dataset.head()

In [ ]:
tempdataset = pd.read_csv("new_file.csv")

fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=False, vertical_spacing=0.001)

x1 = [18,19,20,21,22,23,24]

fig.append_trace(go.Bar(
    x=tempdataset["Do you have Depression?"],
    y=x1,
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(20, 10, 56, 1.0)',
            width=0),
    ),
    name='Depression across the ages',
    orientation='h',
), 1, 1)

fig.append_trace(go.Scatter(
    x=tempdataset["Do you have Panic attack?"], y=x1,
    mode='lines+markers',
    line_color='rgb(40, 0, 128)',
    name='Panic Attacks',
), 1, 2)

fig.update_layout(
    title='Major depression and Panic Attacks',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=True,
        showticklabels=False,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=5,
        domain=[0, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.45],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.47, 1],
        side='top',
        dtick=10000,
    ),
    legend=dict(x=0.029, y=1.038, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []


# Adding labels
for ydn, yd, xd in zip(tempdataset["Do you have Panic attack?"], tempdataset["Do you have Depression?"], x1):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xd, x=ydn+10,
                            text='{:,}'.format(ydn) + '%',
                            font=dict(family='Arial', size=10,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd+10 ,
                            text=str(yd) + '%',
                            font=dict(family='Arial', size=10,
                                      color='rgb(50, 171, 96)'),
                            showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.2, y=-0.109,
                        text="Mental health visualization",
                        font=dict(family='Arial', size=20, color='rgb(150,150,150)'),
                        showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()

In [ ]:
x = ["Appetite change", "Average across symptoms", "Depressed mood", "Difficulty concentrating", "Loss of interest",
    "Low energy", "Low self-esteem", "Psychomotor agitation", "Psychomotor agitation", "Sleep problems", "Suicidal ideation"]

fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=x, y=df3["Nearly every day"], name='Nearly every day',
                         line=dict(color='firebrick', width=4)))
fig.add_trace(go.Scatter(x=x, y=df3["More than half the days"], name = 'More than half the days',
                         line=dict(color='royalblue', width=4)))
fig.add_trace(go.Scatter(x=x, y=df3["Several days"], name='Several days',
                         line=dict(color='black', width=4,
                              dash='dashdot') # dash options include 'dash', 'dot', and 'dashdot'
))

# Edit the layout
fig.update_layout(title='Depressive symptoms across us population',
                   xaxis_title='Entity',
                   yaxis_title='Types of days')


fig.show()

In [ ]:
_ = plt.figure(figsize = (16,5))
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.set_figheight(5)
fig.set_figwidth(20)
plt.subplots_adjust(wspace = 0)
fig.suptitle('Mental Health Disorder in Tech (in the past)', fontsize = 25, y=1.08)

# Pie Chart (Past)
all_techs_past = imp_data[imp_data['tech_flag'] == 1]['mh_disorder_current'].count()
no_past = imp_data[(imp_data['tech_flag'] == 1) & (imp_data['mh_disorder_past'] == 'No')]['mh_disorder_past'].count()
yes_past = imp_data[(imp_data['tech_flag'] == 1) & (imp_data['mh_disorder_past'] == 'Yes')]['mh_disorder_past'].count()
maybe_past = imp_data[(imp_data['tech_flag'] == 1) & (imp_data['mh_disorder_past'] == 'Maybe')]['mh_disorder_past'].count()

labels = 'No', 'Yes', 'Maybe'
sizes = [no_past/all_techs_past, yes_past/all_techs_past, maybe_past/all_techs_past]
colors = ['#73C6B6', '#F0B27A', '#7FB3D5']
explode = (0, 0.03, 0)  # explode 1st slice

ax2.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', shadow=False, startangle=140)
ax2.axis('equal')
ax2.set_title('Overall MH prop% (PAST)', pad = 20, fontsize = 20)

# Barchart (Past)
sns.countplot(x = imp_data[imp_data['tech_flag'] == 1]['country_live'], hue = imp_data['mh_disorder_past'], ax = ax1)
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=15, ha="right")
ax1.set_title('MH by Countries (PAST)', pad = 20, fontsize = 20)
ax1.set_xlabel('Country', fontsize = 18)
ax1.set_ylabel('Count', fontsize = 18)
ax1.legend();

In [ ]:
plt.figure(figsize=(15, 7))
chart = sns.countplot(x = 'Age', data = dataset[dataset['Do you have Depression?'] == 2])
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

In [ ]:
corrmat= dataset.corr()
plt.figure(figsize=(10,10))  
sns.heatmap(corrmat,annot=True, cmap=None)

In [ ]:
#dataset.to_csv('new_file.csv', index=False)

tempdataset.head()

In [26]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping
tf.__version__

'2.10.1'

In [27]:
print("Num GPUs Available: ", len(
    tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [28]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [30]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[21]]


1.0

In [32]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 87.50 %
Standard Deviation: 11.18 %


c:\Users\saif_\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning:

The least populated class in y has only 6 members, which is less than n_splits=10.

